## Exploration of Gradient Descent and Neural Networks

#### Brendan Schlaman (2023.07.01)

**Technical Requirements:**
- Jupyter (with MathJax support for $\LaTeX$)
- `numpy`

**Human Requirements:**
- Reasonable understanding of linear algebra and calculus

---

### Introduction

The purpose of this notebook is to incrementally translate the mathematics of
gradient descent into code in the context of a simple neural network.

We will use only simple math libraries like `numpy` to gain a more fundamental intuition of the concepts; these concepts are
abstracted away in more purpose-built libraries like `pytorch` or TensorFlow.

We will start with the simplest possible neural network: a 1-wide ($m=1$), single layer ($L = 1$) network, and build up from there.

### Notation

The following conventions will be used:

| Variable or symbol | Definition |
|---|---|
| $\mathbf{W}$ | A single weights matrix that feeds into a hidden layer.  $W$ is an $(m \times n)$ matrix, where $n$ is the width (size) of the inputs, and $m$ is the size of the outputs (the width of the hidden layer). |
| $\mathbf{w}$ | Same as above, but when $w$ is a vector (single column matrix).  This is the case where there is a single neuron for a particular layer. |
| `W1` | When using a fixed number of layers *L*, `W1` will be the weights feeding into the first hidden layer (or the output layer).  `Wn` in the code corresponds to $w^{(n)}$ in the math notation. |
| $\mathbf{b}$ | A single bias matrix that contributes to a hidden layer.  $\mathbf{b}$ is a $(m \times 1)$ matrix, where $m$ is the size (width) of the associated hidden layer.  |
| `b1` | When using a fixed number of layers *L*, `b1` will be the weights feeding into the first hidden layer (or the output layer).  `bn` in the code corresponds to $b^{(n)}$ in the math notation. |
| `z` | A preactivated layer.  It is the result of the linear combination (weights times inputs plus bias) before an activation function is applied. |
| `a` | An activated layer, i.e. $\sigma (z)$.  The activated layer becomes the input to the next layer. |
| $L$ | The depth of the neural network; the number of hidden layers plus the output layer. |
| $C_i$ | The cost of the neural network for training example $i$. |
| $j$ | Index of the current layer, the layer denoted in the superscript. |
| $k$ | Index of the previous layer, the layer before the layer denoted in the superscript. |

<br>

> Note that when using a fixed number of layers *L > 1*, the *input layer* is treated as the “0th” layer and does not have wieghts or biases.
Think of this like each layer owning the weights and biases that feed into it.

### Key equations

#### Neural network definitions

$$
    \mathbf{z}^{(L)} = \mathbf{W}^{(L)}\mathbf{a}^{(L-1)} + \mathbf{b}^{(L)}
$$
$$
    \mathbf{a}^{(L)} = \sigma (\mathbf{z}^{(L)})
$$

> Going forward, I'll drop the boldface notation for vectors and matrices.  $W$ is always a matrix, and $b$, $z$, $w$ are always vectors.


#### Computing loss

We'll use the common **Mean Squared Error (MSE)** loss function:

$$
    C_i = \frac{1}{n_L}\sum_{j=0}^{n_L-1}(y_j - a_j^{(L)})^2
$$

where $C_i$ is the loss from a single example, $x_i$.
This is precicely the function we will try to minimize with gradient descent.
Note that $\frac{1}{n_L}$ simply scales the loss and does not change which inputs minimize the function.

We are interested in how the cost of the network changes with respect to it's inputs, $W$ and $b$.
The chain rule gives us our answer:

$$
    \frac{\partial C_i}{\partial W_{jk}^{(l)}} =
        \frac{\partial C_i}{\partial a_j^{(l)}}
        \frac{\partial a_j^{(l)}}{\partial z_j^{(l)}}
        \frac{\partial z_j^{(l)}}{\partial W_{jk}^{(l)}}
$$
$$
    \frac{\partial C_i}{\partial b_j^{(l)}} =
        \frac{\partial C_i}{\partial a_j^{(l)}}
        \frac{\partial a_j^{(l)}}{\partial z_j^{(l)}}
        \frac{\partial z_j^{(l)}}{\partial b_j^{(l)}}
$$

It's also helpful to consider the derivative of $C_i$ with respect to $a_j^{(l)}$,
since through that derivative, we get access future layers' ($l + 1, 2, \dots$) weights and biases.
Notice that the cost function is influenced in $n_{l+1}$ different ways for each $a_j^{(l)}$,
so we use the *multivariable chain rule* to account for all the paths between $a_j^{(l)}$ and the next layer.
Remember that when context switches from $l$ to $l + 1$, the indices $j, k$ are redefined in relation to that new context.

$$
    \frac{\partial C_i}{\partial a_j^{(l)}} =
        \sum_{j=0}^{n_{l+1}-1}
        \frac{\partial C_i}{\partial a_j^{(l+1)}}
        \frac{\partial a_j^{(l+1)}}{\partial z_j^{(l+1)}}
        \frac{\partial z_j^{(l+1)}}{\partial a_j^{(l)}}
$$

The total loss derivative with respect to $W^{(L)}$ is the average of the losses $C_i$ from all examples $x_i$.

$$
    \frac{\partial C}{\partial W^{(L)}} = \frac{1}{n} \sum_{i=0}^{n-1} \frac{\partial C_i}{\partial W^{(L)}}
$$

The full gradient of the loss function over the entire network $\nabla C$ comprises the above derivatives for all layers $l$.

$$
\nabla C =
    \begin{bmatrix}
    \frac{\partial C}{\partial W^{(1)}} \\[1.5ex]
    \frac{\partial C}{\partial b^{(1)}} \\[1.5ex]
    \vdots \\[1.5ex]
    \frac{\partial C}{\partial W^{(L)}} \\[1.5ex]
    \frac{\partial C}{\partial b^{(L)}}
    \end{bmatrix}
$$


Cross entropy loss:

$C = -\sum_{j=1}^{K} y_j \log(\hat{y}_j)$

$C = -\frac{1}{N} \sum_{i=1}^{N} \sum_{j=1}^{K} y_{ij} \log(\hat{y}_{ij})$

#### Nonlinear activation functions

The standard softmax function $\sigma : \mathbb{R}^K \mapsto (0, 1)^K; \; K \geq 1$:

$$
    \sigma(\mathbf{z})_i =
        \frac{e^{z_i}}{\sum_{j=1}^{K} e^{z_j}}
        \text{ for }
        1, \dots, K
        \text{ and }
        \mathbf{z} = (z_1, \dots, z_K) \in \mathbb{R}^K
$$

> The symbols used above are consistent with most online materials,
> where $i$ represents an index of the elements of $\mathbf{z}$,
> not an example index.

For this neural network, we'll be using **ReLU** as our activation function.

### Simpler cases

It's helpful to look at some of the math in edge-case neural networks.

In the case of a single output, our preactivation function can be written as

$$
    z = f(\mathbf{a}) = \mathbf{w} \cdot \mathbf{a} + b
$$

For a single layer ($L = 1$), 1-wide ($m = 1$) network, the partial derivatives of loss are:

\begin{align*}
    \frac{\partial C_i}{\partial w^{(L)}} =&&
        \frac{\partial C_i}{\partial a^{(L)}}
        \frac{\partial a^{(L)}}{\partial z^{(L)}}
        \frac{\partial z^{(L)}}{\partial w^{(L)}}
        &= 2(a^{(L)} - y) \sigma'(z^{(L)}) a^{(L-1)} \\
    \frac{\partial C_i}{\partial b^{(L)}} =&&
        \frac{\partial C_i}{\partial a^{(L)}}
        \frac{\partial a^{(L)}}{\partial z^{(L)}}
        \frac{\partial z^{(L)}}{\partial b^{(L)}}
        &= 2(a^{(L)} - y) \sigma'(z^{(L)}) \\
    \frac{\partial C_i}{\partial a^{(L-1)}} =&&
        \frac{\partial C_i}{\partial a^{(L)}}
        \frac{\partial a^{(L)}}{\partial z^{(L)}}
        \frac{\partial z^{(L)}}{\partial a^{(L-1)}}
        &= 2(a^{(L)} - y) \sigma'(z^{(L)}) w^{(L)}
\end{align*}


### 1-Wide, single layer Neural Network ($L = 1$, $m^{(L)} = 1$)

Start with the simplest NN - an input layer connected to a single output node (one category).

#### Setup

First we'll define some utility functions that help us build the data for our example.
The nature of these functions is chosen arbitrarily.

In [2]:
import random

def init_single_category_data(d: int):
    """Generates random training data with dimensionality `d`.
    The label is also chosen arbitrarily.
    """
    data_size = 10
    labels = [9] * data_size # randomly chose a desired label
    examples = [[random.randrange(-9, 10) for _ in range(d)] for _ in range(data_size)]
    return list(zip(examples, labels))


In [3]:
import numpy as np

input_dimensionality = 3

examples, labels = zip(*init_single_category_data(input_dimensionality))
X, Y = np.array(examples), np.array(labels)

m1 = 1 # the width of our only layer! (remember, we don't count the input layer)

def init_params():
    w1 = np.random.rand(input_dimensionality) * 0.01
    b1 = np.random.randn(m1)
    return w1, b1

w1, b1 = init_params()

X, Y, w1, b1

(array([[-7,  1,  7],
        [ 9, -9, -3],
        [ 0, -4, -1],
        [ 5,  7,  9],
        [-7,  0,  6],
        [ 9, -8, -9],
        [-2, -6, -8],
        [ 4, -4, -3],
        [-9, -2,  7],
        [-7,  1,  5]]),
 array([9, 9, 9, 9, 9, 9, 9, 9, 9, 9]),
 array([-0.00644885, -0.01198742,  0.00288427]),
 array([-0.77612223]))

Forward propagation is simple.  Take the dot product between `w1` and `x` (remember, in this simple case, `w1` and `x` are vectors) and add `b1`.

In [20]:
def forward_prop(
    x: np.ndarray,
    w1: np.ndarray,
    b1: np.ndarray,
):
    z1 = w1.dot(x) + b1
    a1 = max(0, z1) # 1D ReLU
    # print(w1, x, w1.dot(x), z1, a1)
    return z1, a1

def mean_squared_error_loss(label: int, oL: int) -> float:
    return (label - oL)**2

def run(X: np.ndarray, Y: np.ndarray):
    w1, b1 = init_params()
    print(w1, b1)
    for _ in range(1):
        losses = []
        for (i, x) in enumerate(X):
            _, a1 = forward_prop(x, w1, b1)
            losses.append(mean_squared_error_loss(Y[i], a1))
        avg_loss = sum(losses) / len(losses)
        print(losses, avg_loss)

In [21]:
run(X, Y)

[ 0.01310992 -0.0128191  -0.00827641] [-0.20066038]
[ 0.01310992 -0.0128191  -0.00827641] [-7  1  7] -0.16252340793702288 [-0.36318379] 0
[ 0.01310992 -0.0128191  -0.00827641] [ 9 -9 -3] 0.2581904255536273 [0.05753004] [0.05753004]
[ 0.01310992 -0.0128191  -0.00827641] [ 0 -4 -1] 0.05955281037706585 [-0.14110757] 0
[ 0.01310992 -0.0128191  -0.00827641] [5 7 9] -0.0986717655888442 [-0.29933215] 0
[ 0.01310992 -0.0128191  -0.00827641] [-7  0  6] -0.14142789949615908 [-0.34208828] 0
[ 0.01310992 -0.0128191  -0.00827641] [ 9 -8 -9] 0.2950297716810054 [0.09436939] [0.09436939]
[ 0.01310992 -0.0128191  -0.00827641] [-2 -6 -8] 0.11690602260085764 [-0.08375436] 0
[ 0.01310992 -0.0128191  -0.00827641] [ 4 -4 -3] 0.12854531323849513 [-0.07211507] 0
[ 0.01310992 -0.0128191  -0.00827641] [-9 -2  7] -0.15028594963607236 [-0.35094633] 0
[ 0.01310992 -0.0128191  -0.00827641] [-7  1  5] -0.1459705923460966 [-0.34663097] 0
[81, array([79.96776891]), 81, 81, 81, array([79.31025655]), 81, 81, 81, 81] [80

We'll use the same `ReLU` function throughout.

In [ ]:
def ReLU(Z: np.ndarray) -> np.ndarray:
    return np.maximum(0, Z)

#### A beautiful intuition behind the Jacobian

I had a beautifule "eureka" moment today.  I understand the motivation behind the **numerator layout** of the Jacobian matrix.
As a reminder, the Jacobian is defined as follows:

$$
    \mathbf{J} \mathbf{f}(\mathbf{x}) = \nabla \mathbf{f}(\mathbf{x}) = 
    \begin{bmatrix}
    \nabla f_1(\mathbf{x}) \\[1.5ex]
    \nabla f_2(\mathbf{x}) \\[1.5ex]
    \nabla f_3(\mathbf{x}) \\[1.5ex]
    \vdots \\[1.5ex]
    \nabla f_i(\mathbf{x})
    \end{bmatrix}
=
    \begin{bmatrix}
    \frac{\partial}{\partial \mathbf{x}} f_1(\mathbf{x}) \\[1.5ex]
    \frac{\partial}{\partial \mathbf{x}} f_2(\mathbf{x}) \\[1.5ex]
    \frac{\partial}{\partial \mathbf{x}} f_3(\mathbf{x}) \\[1.5ex]
    \vdots \\[1.5ex]
    \frac{\partial}{\partial \mathbf{x}} f_i(\mathbf{x})
    \end{bmatrix}
= 
    \begin{bmatrix}
    \frac{\partial}{\partial x_1} f_1(\mathbf{x}) \frac{\partial}{\partial x_2} f_1(\mathbf{x}) \dots \frac{\partial}{\partial x_j} f_1(\mathbf{x}) \\[1.5ex]
    \frac{\partial}{\partial x_1} f_2(\mathbf{x}) \frac{\partial}{\partial x_2} f_2(\mathbf{x}) \dots \frac{\partial}{\partial x_j} f_2(\mathbf{x}) \\[1.5ex]
    \frac{\partial}{\partial x_1} f_3(\mathbf{x}) \frac{\partial}{\partial x_2} f_3(\mathbf{x}) \dots \frac{\partial}{\partial x_j} f_3(\mathbf{x}) \\[1.5ex]
    \vdots \\[1.5ex]
    \frac{\partial}{\partial x_1} f_i(\mathbf{x}) \frac{\partial}{\partial x_2} f_i(\mathbf{x}) \dots \frac{\partial}{\partial x_j} f_i(\mathbf{x})
    \end{bmatrix}
$$